In [13]:
import base64
import requests

# OpenAI API Key
api_key = "xxxxx"

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [14]:
import pandas as pd

base_path = "clear/apartment_images/"
gt_path = 'ground_truth.csv'

# convert list of images to path
def image_list(folder_id, image_string):
    images = []
    array = image_string.split(';')
    for item in array:
        image_url = base_path + str(folder_id) + '/image_' + item + '.jpg'
        images.append(image_url)
    return images


In [2]:
# get training ids
import random

energy_0_ids = [30, 44]
energy_0_training_sample = random.sample(energy_0_ids, 1) 
print(energy_0_training_sample)

[30]


In [80]:
energy_100_ids = [2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 29, 31, 32, 33, 35, 36, 37, 38, 40, 41, 42, 43, 45, 46]

energy_100_training_sample = random.sample(energy_100_ids, 21) 
print(energy_100_training_sample)

[7, 32, 10, 46, 43, 31, 29, 3, 9, 45, 12, 21, 40, 13, 2, 16, 11, 17, 15, 36, 8]


In [6]:
energy_75_ids = [0, 39]
energy_75_training_sample = random.sample(energy_75_ids, 1) 
print(energy_75_training_sample)

[39]


In [74]:
energy_misc_ids = [1, 4, 14, 19, 26, 27, 28, 34]
energy_misc_training_sample = random.sample(energy_misc_ids, 5) 
print(energy_misc_training_sample)

[26, 19, 14, 28, 1]


In [81]:
# Combine all lists and check for duplicates, there shouldn't be!
training_samples_combined = energy_0_training_sample + energy_100_training_sample + energy_75_training_sample + energy_misc_training_sample
duplicates = [item for item in set(training_samples_combined) if training_samples_combined.count(item) > 1]
print("Duplicates:", duplicates)

Duplicates: []


In [85]:
# Sort them by increasing id
training_sample = sorted(training_samples_combined)
print(training_sample)
if len(training_sample) != 28:
    print("insufficient training sample", len(training_sample))
else:
    print("training sample length", len(training_sample))

[1, 2, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 21, 26, 28, 29, 30, 31, 32, 36, 39, 40, 43, 45, 46]
training sample length 28


In [19]:
from openai import OpenAI
client = OpenAI(api_key=api_key)
def get_completion(prompt):
    response = client.chat.completions.create(
      model="gpt-4o-2024-08-06", #"gpt-4o",
      messages=[
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": prompt,
            },
          ],
        }
      ],
      max_tokens=4000,
    )
    # print(response)
    return response.choices[0].message.content

In [23]:
# choose 3 buildings to extract feature from

# randomly pick one from the 0% training sample, there is just 1
item1 = random.sample(energy_0_training_sample, 1)
print('item1', item1)

item1 [30]


In [24]:
# randomly pick one from the 100% training sample
item2 = random.sample(energy_100_training_sample, 1)
print('item2', item2)

item2 [7]


In [26]:
# randomly pick one from the misc training sample
item3 = random.sample(energy_misc_training_sample, 1)
print('item3', item3)

item3 [26]


In [62]:
custom_prompt = "Your task is to provide a detailed label of every visible feature in the images relating to artificial lights for the building that will help determine the type of lighting whether it is no low energy lighting, low energy in 20%, low energy in 40%, low energy in 60%, low energy in 80%, low energy in 100%. List 50 visible features that are significant for determining the type of bulbs used in the lights. Don't explain the label."
image_type = 'Lighting image' # change this for age, window etc. this is used to determine which images to use

def llm_get_features(images):

    prompt = f"""

    You are a surveyor. You are given a set of images that belong to the same building.
    
    {custom_prompt}
    
    The building is located in UK. Return the features as a list.
    
    """

    content = [
            {
              "type": "text",
              "text": prompt
            }
          ]

    for image in images:
        base64_image = encode_image(image)
        content.append({
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
            }
        })

    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }

    payload = {
      "model": "gpt-4o",
      "messages": [
        {
          "role": "user",
          "content": content
        }
      ],
      "max_tokens": 5000
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # print(response.text)
    print(response.json()['choices'][0]['message']['content'])


def extract_features(row_id):
    df = pd.read_csv(gt_path)
    raw_image_list = df.loc[row_id][image_type]
    print('\n\n\n---row', row_id, df.loc[row_id]['Address'], df.loc[row_id]['Thesquare Building URL'])
    llm_get_features(image_list(row_id, raw_image_list))

In [63]:
extract_features(30)




---row 30 Lawrence House , 238 City Road , EC1V 2PQ https://www.thesqua.re/london/serviced-apartments/lawrence-house-serviced-apartment-882227696
1. Ceiling light fixtures
2. Wall sconces
3. Table lamps
4. Floor lamps
5. Pendant lights
6. Light bulbs visible
7. Bulb shape
8. Bulb size
9. Lamp shade design
10. Lamp base material
11. LED presence
12. Fluorescent tube
13. CFL bulbs
14. Halogen bulbs
15. Incandescent bulbs
16. Dimmable lights
17. Smart lighting system
18. Light switch design
19. Dimmer switch
20. Number of light sources
21. Light distribution pattern
22. Overhead lighting intensity
23. Ambient lighting
24. Downlighting presence
25. Uplighting presence
26. Accent lighting
27. Task lighting
28. Mood lighting
29. Color temperature
30. Light intensity/brightness
31. Light reflection on surfaces
32. Shadow formation
33. Light spread angle
34. Light fixture material
35. Modern lighting design
36. Retro lighting design
37. Energy label indication
38. Light fixture size
39. Lig

In [64]:
extract_features(7)




---row 7 50 Bruges Place , 15-20 Randolph Street , NW1 0TL https://www.thesqua.re/london/serviced-apartments/camden-road-serviced-apartments-by-mysquare-1576707764
1. Ceiling light fixtures
2. Recessed lighting
3. Pendant lights
4. Wall sconces
5. Under-cabinet lighting
6. Track lighting fixture
7. Table lamps
8. Floor lamps
9. LED bulb usage
10. CFL bulb usage
11. Incandescent bulb usage
12. Halogen bulb usage
13. LED strip lighting
14. Dimmer switches
15. Motion sensor lights
16. Smart bulbs
17. Light fixture design
18. Light fixture material
19. Color of light emitted
20. Light fixture placement
21. Number of bulbs in fixtures
22. Light fixture size
23. Light fixture shape
24. Lamp shade presence
25. Lamp shade material
26. Lamp shade color
27. Light switch type
28. Battery-powered lights
29. Solar-powered lights
30. Reflective surfaces
31. Skylights or natural light usage
32. Lighting control systems
33. Ceiling fan lights
34. Chandelier presence
35. Energy-saving labels
36. Fix

In [65]:
extract_features(26)




---row 26 Flat 604,25 Indescon Square, E14 9DG , nan , E14 9DG https://www.thesqua.re/london/serviced-apartments/indescon-square-at-canary-wharf-by-mysquare-1205007705
1. Ceiling spotlights
2. Lamp shades style
3. Number of bulbs per fixture
4. Light fixture design
5. Light color temperature
6. Bulb shape
7. Light distribution pattern
8. Pendant lighting presence
9. Recessed light fittings
10. Dimmer switches presence
11. Wall light fixtures
12. Table lamps
13. Floor lamps
14. LED signage
15. Lighting control systems
16. Presence of chandeliers
17. Skylights with artificial lighting
18. Reflective surfaces
19. Window placement related to lighting
20. Lighting fixture material
21. Energy efficiency labels on bulbs
22. Smart bulb presence
23. Built-in under-cabinet lighting
24. Electric socket proximity to lighting
25. Multiple light sources in a room
26. Light diffusers usage
27. Glass fixtures
28. Open filament bulbs
29. Light fixture mount type
30. Use of lampshades
31. Track light

In [66]:
features_1 = [
    "Ceiling light fixtures",
    "Wall sconces",
    "Table lamps",
    "Floor lamps",
    "Pendant lights",
    "Light bulbs visible",
    "Bulb shape",
    "Bulb size",
    "Lamp shade design",
    "Lamp base material",
    "LED presence",
    "Fluorescent tube",
    "CFL bulbs",
    "Halogen bulbs",
    "Incandescent bulbs",
    "Dimmable lights",
    "Smart lighting system",
    "Light switch design",
    "Dimmer switch",
    "Number of light sources",
    "Light distribution pattern",
    "Overhead lighting intensity",
    "Ambient lighting",
    "Downlighting presence",
    "Uplighting presence",
    "Accent lighting",
    "Task lighting",
    "Mood lighting",
    "Color temperature",
    "Light intensity/brightness",
    "Light reflection on surfaces",
    "Shadow formation",
    "Light spread angle",
    "Light fixture material",
    "Modern lighting design",
    "Retro lighting design",
    "Energy label indication",
    "Light fixture size",
    "Light bulb fitting",
    "Energy-efficient design",
    "Energy star rating",
    "Fixture mounting type",
    "Energy usage markings",
    "Estimated bulb lifespan",
    "Energy consumption labels",
    "Lighting zone plans",
    "Presence of control systems",
    "Motion-sensor lights",
    "Eco-friendly packaging",
    "Manufacturer's label on lights"
]

In [67]:
features_2 = [
    "Ceiling light fixtures",
    "Recessed lighting",
    "Pendant lights",
    "Wall sconces",
    "Under-cabinet lighting",
    "Track lighting fixture",
    "Table lamps",
    "Floor lamps",
    "LED bulb usage",
    "CFL bulb usage",
    "Incandescent bulb usage",
    "Halogen bulb usage",
    "LED strip lighting",
    "Dimmer switches",
    "Motion sensor lights",
    "Smart bulbs",
    "Light fixture design",
    "Light fixture material",
    "Color of light emitted",
    "Light fixture placement",
    "Number of bulbs in fixtures",
    "Light fixture size",
    "Light fixture shape",
    "Lamp shade presence",
    "Lamp shade material",
    "Lamp shade color",
    "Light switch type",
    "Battery-powered lights",
    "Solar-powered lights",
    "Reflective surfaces",
    "Skylights or natural light usage",
    "Lighting control systems",
    "Ceiling fan lights",
    "Chandelier presence",
    "Energy-saving labels",
    "Fixture age or modernity",
    "Multiple light settings",
    "Decorative lighting elements",
    "Ambient light level",
    "Task lighting presence",
    "Accent lighting presence",
    "Emergency lighting fixtures",
    "Lighting control apps",
    "Flicker presence",
    "Smart home integration",
    "Cost indication on fixtures",
    "Brand of bulb",
    "Presence of timers",
    "Adjustable light direction",
    "Bulb accessibility and replacement"
]

In [68]:
features_3 = [
    "Ceiling spotlights",
    "Lamp shades style",
    "Number of bulbs per fixture",
    "Light fixture design",
    "Light color temperature",
    "Bulb shape",
    "Light distribution pattern",
    "Pendant lighting presence",
    "Recessed light fittings",
    "Dimmer switches presence",
    "Wall light fixtures",
    "Table lamps",
    "Floor lamps",
    "LED signage",
    "Lighting control systems",
    "Presence of chandeliers",
    "Skylights with artificial lighting",
    "Reflective surfaces",
    "Window placement related to lighting",
    "Lighting fixture material",
    "Energy efficiency labels on bulbs",
    "Smart bulb presence",
    "Built-in under-cabinet lighting",
    "Electric socket proximity to lighting",
    "Multiple light sources in a room",
    "Light diffusers usage",
    "Glass fixtures",
    "Open filament bulbs",
    "Light fixture mount type",
    "Use of lampshades",
    "Track lighting",
    "Mood lighting elements",
    "Articulated arms in fixtures",
    "Backlighting",
    "Cove lighting",
    "Sconce lighting",
    "Lighting in alcoves",
    "Staircase lighting",
    "Kitchen under-shelf lighting",
    "Accent lighting",
    "Uplighting features",
    "Colour rendering index (CRI)",
    "Presence of coloured bulbs",
    "Light beam angle",
    "Usage of frosted bulbs",
    "Bulb wattage indication",
    "Brand labeling on fixtures",
    "Timer-based lighting system",
    "Ceiling fan with lights",
    "Exterior lighting visibility"
]

In [71]:
raw_feature_list = features_1 + features_2 + features_3

prompt =  f"""
I have a list of features: {raw_feature_list}. First, remove duplicated items, including features semantically similar. Then cluster these features based on the type of feature. Aim to produce 8 clusters. 
"""

result = get_completion(prompt)
print(result)

To produce a coherent list of unique and clustered lighting features, let's first remove duplicates and semantically similar items. After that, we'll cluster them into meaningful categories. Here's the refined and clustered list:

### Cluster 1: Fixture Types & Placement
- Ceiling light fixtures
- Wall sconces
- Table lamps
- Floor lamps
- Pendant lights
- Recessed lighting
- Under-cabinet lighting
- Track lighting fixture
- Chandelier presence
- Ceiling spotlights
- Ceiling fan lights
- Wall light fixtures
- Built-in under-cabinet lighting
- Staircase lighting
- Kitchen under-shelf lighting

### Cluster 2: Bulb Types & Characteristics
- Light bulbs visible
- Bulb shape
- Bulb size
- LED presence / LED bulb usage
- CFL bulbs / CFL bulb usage
- Halogen bulbs / Halogen bulb usage
- Incandescent bulbs / Incandescent bulb usage
- Open filament bulbs
- Smart bulbs
- LED strip lighting

### Cluster 3: Light Control & Systems
- Smart lighting system
- Light switch design
- Dimmer switch / Dim

In [72]:
categories = f"""
### Cluster 1: Fixture Types & Placement
- Ceiling light fixtures
- Wall sconces
- Table lamps
- Floor lamps
- Pendant lights
- Recessed lighting
- Under-cabinet lighting
- Track lighting fixture
- Chandelier presence
- Ceiling spotlights
- Ceiling fan lights
- Wall light fixtures
- Built-in under-cabinet lighting
- Staircase lighting
- Kitchen under-shelf lighting

### Cluster 2: Bulb Types & Characteristics
- Light bulbs visible
- Bulb shape
- Bulb size
- LED presence / LED bulb usage
- CFL bulbs / CFL bulb usage
- Halogen bulbs / Halogen bulb usage
- Incandescent bulbs / Incandescent bulb usage
- Open filament bulbs
- Smart bulbs
- LED strip lighting

### Cluster 3: Light Control & Systems
- Smart lighting system
- Light switch design
- Dimmer switch / Dimmer switches
- Motion-sensor lights / Motion sensor lights
- Lighting control systems / Lighting control apps
- Smart home integration
- Presence of timers / Timer-based lighting system
- Adjustable light direction

### Cluster 4: Lighting Design & Style
- Lamp shade design
- Lamp base material
- Light fixture design
- Light fixture material
- Modern lighting design
- Retro lighting design
- Decorative lighting elements
- Lamp shades style
- Light fixture shape
- Articulated arms in fixtures
- Lamp shade material
- Lamp shade color

### Cluster 5: Energy Efficiency & Labels
- Energy-efficient design
- Energy star rating
- Energy label indication
- Energy usage markings
- Estimated bulb lifespan
- Energy consumption labels
- Energy-saving labels
- Bulb wattage indication
- Eco-friendly packaging

### Cluster 6: Light Distribution & Qualities
- Light distribution pattern
- Light intensity/brightness
- Ambient lighting
- Overhead lighting intensity
- Downlighting presence
- Uplighting presence
- Accent lighting / Accent lighting presence
- Task lighting / Task lighting presence
- Mood lighting / Mood lighting elements
- Colour temperature / Light color temperature
- Reflective surfaces
- Shadow formation
- Light spread angle
- Light reflection on surfaces
- Light beam angle
- Usage of frosted bulbs
- Colour rendering index (CRI)

### Cluster 7: Additional Design & Feature Attributes
- Fixture mounting type
- Fixture age or modernity
- Multiple light settings
- Ambient light level
- Emergency lighting fixtures
- Light fixture size / Number of bulbs per fixture
- Fixture mount type

### Cluster 8: Advanced & Specialized Features
- Skylights or natural light usage with artificial lighting
- Presence of chandeliers
- Light diffusers usage
- Glass fixtures
- Cove lighting
- Backlighting
- Sconce lighting
- Lighting in alcoves
- Usage of frosted bulbs
- Lighting zone plans
- Electric socket proximity to lighting
- Exterior lighting visibility
"""

prompt =  f"""
Given this list {categories}, first clean the list to contain text only, then produce a python array, each subarray for each category.
"""

result = get_completion(prompt)
print(result)

Here's a cleaned version of the list, converted into a Python array format, with each cluster represented as a separate subarray:

```python
lighting_features = [
    # Cluster 1: Fixture Types & Placement
    [
        "Ceiling light fixtures",
        "Wall sconces",
        "Table lamps",
        "Floor lamps",
        "Pendant lights",
        "Recessed lighting",
        "Under-cabinet lighting",
        "Track lighting fixture",
        "Chandelier presence",
        "Ceiling spotlights",
        "Ceiling fan lights",
        "Wall light fixtures",
        "Built-in under-cabinet lighting",
        "Staircase lighting",
        "Kitchen under-shelf lighting"
    ],
    # Cluster 2: Bulb Types & Characteristics
    [
        "Light bulbs visible",
        "Bulb shape",
        "Bulb size",
        "LED presence / LED bulb usage",
        "CFL bulbs / CFL bulb usage",
        "Halogen bulbs / Halogen bulb usage",
        "Incandescent bulbs / Incandescent bulb usage",
        "Open